In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ipynb.fs.full.Funciones_basicas import *
from ipynb.fs.full.Algoritmos import *
from ipynb.fs.full.Visualizaciones import *
import numpy as np

In [2]:
path = '../data/graph_geom_corrected_cycles.csv'
csv_grafo = pd.read_csv(path, sep=';')

geometry = gpd.read_file(path, GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

path_pesos = '../data/pesos_tapas.csv'
pesos = pd.read_csv(path_pesos)

out = {1003950, 15004, 15131, 1003744, 15190, 1003746, 13730, 1003167, 13731, 13732, 13733, 14062, 1003206, 16503, 13735, 16094, 16095}

S = set()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        S.add(origin)
        S.add(dest)

S = list(S)
id_ = {}; _id = {}; N = 0
for u in S:
    id_[u] = N; _id[N] = u
    N += 1

G = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        G.add_edge(id_[origin], id_[dest])
        
path = '../data/corrected_dijkstra.csv'
csv_grafo = pd.read_csv(path, sep=',')

T = nx.DiGraph()
for index, row in csv_grafo.iterrows():
    origin = row['self']
    dest = row['other']
    if origin not in out and dest not in out:
        T.add_edge(id_[origin], id_[dest])

W = [0] * N

for u in range(N):
    if pesos[pesos['ID_tapa'] == _id[u]].shape[0] >= 1:  ## tomamos primera columna con el id, si no hay peso = 0
        W[u] = pesos[pesos['ID_tapa'] == _id[u]].iloc[0]['per_predio']

In [ ]:
E = []
for u in range(N):
    E.append((get_size(G, [0] * N, u), u))

E.sort()
print(E)

In [6]:
def restrict_graph(G, V, u):
    I = set(get_ideal(G, V, u))
    
    newG = nx.DiGraph()
    
    for u, v in G.edges:
        if (u in I) and (v in I):
            newG.add_edge(u, v)
    
    return newG

In [39]:
newG = restrict_graph(G, [0] * N, 4000)
topo = [u for u in nx.algorithms.dag.topological_sort(newG)]
print(topo)

newN = len(topo)

[4067, 2479, 2513, 4072, 1117, 1116, 1115, 923, 1122, 1121, 4069, 4070, 1111, 4020, 4411, 4410, 4019, 4021, 4013, 4006, 4005, 4023, 4022, 1837, 4058, 4057, 4068, 919, 4003, 2093, 4004, 2092, 4421, 4002, 2091, 4024, 4025, 4026, 4001, 4000]


In [40]:
map_prev = {}; next_map = {}
limK = 5

def minimize(id_, ss, k):
    
    u = topo[id_]
    
    if (V_bits & (1 << u)):
        return minimize(id_ + 1, V_bits)
    
    if id_ == newN:
        return 0
    
    if (u, V_bits, k) in map_prev.keys():
        return map_prev[(u, V_bits, k)]
    
    V = [0] * N
    for i in range(N):
        V[i] = (V_bits & (1 << i))
        
    I = get_ideal(newG, V, u)
    if id_ == newN - 1:
        return len(I) ** 2
    
    V_bits_new = V_bits
    for v in I:
        V_bits_new |= (1 << v)
    
    ans = minimize(id_ + 1, V_bits, k)
    if k < limK - 1:
        ans = min(ans, len(I) ** 2 + minimize(id_ + 1, V_bits_new, k + 1))
    map_prev[(u, V_bits, k)] = ans
    return ans

In [41]:
sys.setrecursionlimit(10000)

I = get_ideal(G, [0] * N, 4000)
V_bits = 0
for u in range(N):
    if u not in I:
        V_bits |= (1 << u)

minimize(0, 0, 0)

KeyboardInterrupt: 

In [ ]:
# # Esta función usa programación dinámica para encontrar las muestras

# def get_dp_table(G, V, k, s, mult):
    
#     N = len(V)

#     order = get_ideal(G, V, s)[::-1]  # Lista de nodos en ideal a trabajar desde más profundo a menos profundo
#     size = len(order)
    
#     # inicializamos tablas

#     # Combinaciones de (número de muestras, nodos sin muestrear, ideal máximo, ideal mínimo) en el ideal de cada u
#     possibilities = {u: [] for u in order}
    
#     # Punteros óptimos para generar cada posibilidad en la tabla (posibilities) para cada u
#     next_dicts = {u: {} for u in order}

#     for u in order: # Para cada nodo de más a menos profundo

#         new_possibilities = {}; new_next = {}  # Subtablas del nodo u (al final se pasan a tablas finales)
#         predecessors = [v for v in G.predecessors(u) if not V[v]]  # nodos hijos en el árbol

#         # calculamos cantidad de combinaciones en posibilidades de hijos
#         # y generamos tabla (mul) para generar las combinaciones
        
#         mul = [1 for v in predecessors]; combs = 1 if predecessors else 0
#         for i in range(len(predecessors)):
#             now = len(possibilities[predecessors[i]])
#             for j in range(i):
#                 mul[j] *= now
#             combs *= now
        
#         for i in range(combs):  # por cada combinación de posibilidades

#             indexes = [0 for v in predecessors]  # generamos la combinación
#             for j in range(len(predecessors)):
#                 indexes[j] = (i // mul[j])
#                 i %= mul[j]
                
#             # calculamos la posibilidad que genera esta combinación

#             l = 0; size_left = 1; max_ideal = 0; min_ideal = N
#             for j in range(len(predecessors)):
#                 pos = possibilities[predecessors[j]][indexes[j]]
#                 l += pos[0]
#                 size_left += pos[1]
#                 max_ideal = max(max_ideal, pos[2])
#                 min_ideal = min(min_ideal, pos[3])

#             if l >= k:
#                 continue
                
#             # Agregamos posibilidades

#             if new_possibilities.get((l, max_ideal // mult)):
#                 prev = new_possibilities[(l, max_ideal // mult)]
#                 if size_left < prev[0] or (size_left == prev[0] and min_ideal > prev[2]):
#                     new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
#                     new_next[(l, max_ideal // mult)] = indexes
#             else:
#                 new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
#                 new_next[(l, max_ideal // mult)] = indexes
                
#             if l < k:
#                 if new_possibilities.get((l + 1, max(size_left, max_ideal) // mult)):
#                     prev = new_possibilities[(l + 1, max(size_left, max_ideal) // mult)]
#                     if 0 < prev[0] or (0 == prev[0] and min(size_left, min_ideal) > prev[2]):
#                         new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
#                         new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes
#                 else:
#                     new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
#                     new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes

#         # Caso especial, si no hay hijos no hay combinaciones y agregamos posibilidades de muestrear o no
                    
#         if combs == 0:
#             new_possibilities[(0, 0)] = (1, 0, N)
#             new_next[(0, 0)] = []
#             new_possibilities[(1, 0)] = (0, 1, 1)
#             new_next[(1, 0)] = []
        
#         final_possibilities = set()
#         for item in new_possibilities.items():
#             o, b = item;
#             a = (o[0], b[0], b[1], b[2])
#             final_possibilities.add(a)
#             next_dicts[u][a] = new_next[o]

#         possibilities[u] = list(final_possibilities)

#     return possibilities, next_dicts